# Lens Data Science Assessment

Context
---

Lens Protocol is a composable and decentralized social graph. It is a powerful tool to empower web3 user adoption through creativity & connexion.
POAP are digital mementos, minted in celebration of life's remarkable moments. User can collect tokenized experiences and directly have an on-chain proof of attendance.

### Libraries & Utilitaries

In [3]:
import pandas as pd
import os
import time
import requests
import json
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
class APIBuilder:
    """
    Basic API Module
    """
    def __init__(self, url, headers):
        self.url = url
        self.headers = headers

    def get(self, query):
        get_url = self.url+query
        request = requests.get(get_url, headers=self.headers)
        return request


class PoapLoader:
    """
    Recursive POAP collection loader
    """
    def __init__(self):
        print('⏳ Poap events with holders retrieval started...')
        self.eth_subgraph = "https://api.thegraph.com/subgraphs/name/poap-xyz/poap"
        self.xdai_subgraph = "https://api.thegraph.com/subgraphs/name/poap-xyz/poap-xdai"
        self.extracted_data = []
        self.get_poap_token_data()
        print('✨ Poap events with holders successfully loaded !')

    def get_poap_token_data(self):
        token_id = 0
        query = """query get_events($token_id: ID = "0") {
            events(where: {id_gt: $token_id, tokenCount_gt: "10"}, first: 200) {
                tokens {
                owner {
                    id
                }
                }
                id
                created
                tokenCount
            }
            }
        """
        while True:
            gql_request = requests.post(
                self.xdai_subgraph,
                json={
                    "query": query,
                    "variables": {"token_id": token_id},
                }
            )
            if gql_request.status_code != 200 or "errors" in gql_request.text:
                time.sleep(2)
                continue
            current_data = json.loads(gql_request.text)
            if not current_data["data"]["events"]:
                return
            for raw_event_data in current_data["data"]["events"]:
                event_data = {}
                event_data['event_id'] = raw_event_data['id']
                event_data['token_count'] = raw_event_data['tokenCount']
                event_data['holders'] = list(map(lambda holder: holder['owner']['id'].lower(), raw_event_data['tokens']))
                self.extracted_data.append(event_data)
                token_id = int(event_data['event_id'])

### 1 - Finding the top 10 POAPs thatt are most frequently held by Lens users

As a first look, we have multiple ways to approach this. I personnally think gathering the 80k+ addresses holding a lens profile & checking whether they hold any POAPs is more convenient.
We will be using Moralis API to retrieve all lens Holders because it saves us a lot of time & a subgraph of The Graph to query Poap events

An important point is to ensure address format while comparison, let's simply ensure all addresses are `.lower()`

Resources : 

- Moralis API
- https://api.poap.xyz/events
- https://api.thegraph.com/subgraphs/name/poap-xyz/poap-xdai/graphql

In [5]:
LENS_PROFILE_ADDRESS = '0xdb46d1dc155634fbc732f92e853b10b288ad5a1d'
moralis_url = 'https://deep-index.moralis.io/api/v2/'
query = f'nft/{LENS_PROFILE_ADDRESS}/owners?chain=polygon&format=decimal'
headers = {
    'Accept': '*/*',
    'Connection': 'keep-alive',
    'x-api-key': os.environ.get('API_KEY')
}

In [6]:
def get_lens_holders():
    """
    Retrieves and Stores lens holders addresses
    """
    print(f'⏳ Lens users retrieval started...')
    api = APIBuilder(moralis_url, headers)
    lens_holders = []
    cursor = None
    while True:
        if cursor:
            cursor_query = query+f'&cursor={cursor}'
            res = api.get(cursor_query)
        else:
            res = api.get(query)
        if res.status_code != 200:
            time.sleep(2)
        res = json.loads(res.text)
        holders = res.get('result')
        try:
            next_holders = list(map(lambda holder:  holder.lower() if type(holder) == 'str' else holder['owner_of'].lower(), holders))
            lens_holders += next_holders
            cursor = res.get('cursor')
            if not cursor:
                lens_df = pd.DataFrame(lens_holders)
                lens_df.to_csv('lens_holders.csv')
                print('✨ Lens users acquired !')
                return lens_holders
        except Exception as e:
            print(e)
            return lens_holders


In [7]:
lens_holders = get_lens_holders()

⏳ Lens users retrieval started...
✨ Lens users acquired !


In [8]:
lens_holders = pd.DataFrame(lens_holders, columns=['addresses'])
lens_holders = lens_holders.drop_duplicates(subset=['addresses'])
list_lens_users = list(lens_holders.addresses)

In [9]:
lens_holders.head()

,addresses
0,0x439b947e7613399587649e9ab96efffdd8c94d55
1,0x965599f319521b86eecaa5414fb37811d11794dc
2,0x0b55836bc1f8c331c6da54f670229047720f59f3
3,0x4f04a5cc46b4eb625f43a1e94c0eacf19f446f8e
4,0xecb072e56fba965c1447ea4174d20f397aa88db1


Let's now load POAP events using The Graph API

In [10]:
poap_loader = PoapLoader()

⏳ Poap events with holders retrieval started...
✨ Poap events with holders successfully loaded !


In [11]:
poap_events = poap_loader.extracted_data
len(poap_events)

25387

At this points, we have 90k **unique** Lens handle users & more than 25k POAP events to analyse.
This is quite fun knowing we will have to loop through each of these events and get the number of Lens holders.

We have 3 options : list comprehensions, bitwise operation and intersections. Since we will have $Size(Poap_e) < Size(List_{lens})$, we can use intersections because it's more readable, flexible and faster

In [12]:
for event_index, event in enumerate(poap_events):
    event_lens_user_holders = frozenset(event['holders']).intersection(list_lens_users)
    event_count_lens_users = len(event_lens_user_holders)
    poap_events[event_index]["count_lens_users"] = event_count_lens_users

In [13]:
poap_events_df = pd.DataFrame(poap_events)

In [14]:
poap_events_df = poap_events_df.drop(columns=['holders'])
poap_events_df.head()

,event_id,token_count,count_lens_users
0,10002,1798,22
1,10005,44,7
2,10017,998,27
3,10018,67,4
4,1003,12,1


Let's now sort out DataFrame by lens users and map these events id with the dedicated Poap event information.

In [15]:
top_10_poap_held_by_lens_users = poap_events_df.sort_values(by=['count_lens_users'], ascending=False).iloc[:10]
top_10_poap_held_by_lens_users['event_id'] = top_10_poap_held_by_lens_users['event_id'].astype('int64')

In [16]:
top_10_poap_held_by_lens_users

,event_id,token_count,count_lens_users
15918,49299,4491,99
17340,53352,138,96
15494,48089,146,96
17342,53358,139,95
20413,62102,777,92
15574,48299,201,92
23991,75982,766,90
18928,57801,2038,89
17828,54726,1756,89
22585,70142,1941,88


In [17]:
headers = {
    'Accept': '*/*',
    'Connection': 'keep-alive'
}

poap_api = APIBuilder("https://api.poap.xyz/events", headers)
events = poap_api.get('')

In [18]:
events = json.loads(events.text)
events_df = pd.DataFrame(events)
events_df.to_csv('poap_events.csv')

In [19]:
events_df.join(top_10_poap_held_by_lens_users.set_index('event_id'), on='id', how='right')

,id,fancy_id,name,event_url,image_url,country,city,description,year,start_date,end_date,expiry_date,from_admin,virtual_event,event_template_id,event_host_id,private_event,token_count,count_lens_users
27681,49299,lensterxyz-early-bloomer-2022,Lenster.xyz Early Bloomer,https://lenster.xyz,https://assets.poap.xyz/lensterxyz-early-bloom...,,,A POAP for all the early users of Lenster.xyz\r\n,2022,11-Jun-2022,11-Jun-2022,11-Jun-2023,False,True,0,0,False,4491,99
23754,53352,developer-dao-developer-dao-improvement-propos...,Developer DAO - Developer DAO Improvement Prop...,https://www.developerdao.com/,https://assets.poap.xyz/developer-dao-develope...,,,This POAP is rewarded to Developer DAO members...,2022,01-Jul-2022,01-Jul-2022,01-Aug-2022,False,True,0,0,False,138,96
28576,48089,first-zkex-trivia-competition-2022,First ZKEX Trivia Competition,,https://assets.poap.xyz/first-zkex-trivia-comp...,,,This drop is for participating in the ZKEX Tri...,2022,06-Jun-2022,06-Jun-2022,06-Jul-2022,False,True,0,0,False,146,96
23490,53358,developer-dao-vote-p15-2022,Developer DAO - Vote P15,https://www.developerdao.com/,https://assets.poap.xyz/developer-dao-vote-p15...,,,This POAP is rewarded to Developer DAO members...,2022,04-Jul-2022,04-Jul-2022,04-Aug-2022,False,False,0,0,False,139,95
15163,62102,the-merge-poap-2022,The Merge POAP,https://infura.io/,https://assets.poap.xyz/the-merge-poap-2022-lo...,,,This is a POAP for attending our Merge Weekly ...,2022,26-Aug-2022,26-Aug-2022,19-Sep-2022,False,False,0,0,False,777,92
28943,48299,developer-dao-increase-short-term-allocation-p...,Developer DAO - Increase Short-Term Allocation...,https://www.developerdao.com/,https://assets.poap.xyz/developer-dao-increase...,,,This POAP is rewarded to DAO members who voted...,2022,03-Jun-2022,03-Jun-2022,03-Jul-2022,False,True,0,0,False,201,92
1108,75982,zerion-connect-2324-2022,Zerion Connect #24,https://zerion.io/discord,https://assets.poap.xyz/zerion-connect-2324-20...,,,Topics for this week:\r\n-Product updates\r\n-...,2022,13-Oct-2022,13-Oct-2022,03-Nov-2022,False,True,0,0,False,766,90
18928,57801,zerion-connect-18-2022,Zerion Connect #18,https://zerion.io/discord,https://assets.poap.xyz/zerion-connect-18-2022...,,,Topics for this week:\r\n-Product updates\r\n-...,2022,04-Aug-2022,04-Aug-2022,18-Aug-2022,False,True,0,0,False,2038,89
22035,54726,zerion-connect-15-2022,Zerion Connect #15,https://zerion.io/discord,https://assets.poap.xyz/zerion-connect-15-2022...,,,Topics for this week:\r\n-Product updates\r\n-...,2022,14-Jul-2022,14-Jul-2022,14-Aug-2022,False,True,0,0,False,1756,89
9675,70142,zerion-connect-23-2022,Zerion Connect #23,https://zerion.io/discord,https://assets.poap.xyz/zerion-connect-23-2022...,,,Topics for this week:\r\n-Product updates\r\n-...,2022,29-Sep-2022,29-Sep-2022,20-Oct-2022,False,True,0,0,False,1941,88


And here we are ! We have been able to collect the top 10 Poap events that are most held by Lens users. Undoubtly the first place is for Lens Early Bloomer events which is actually very logical. Also we are able to see that most of them are Virtual events, and interactions can be easier ! 

Let's compare these results with the provided sample of lens users. We still must ensure correct address format.
However, with this small sample, result might be very different since it could be not representative of the overall population.

In [20]:
lens_user_sample = pd.read_csv('profiles_sample.csv')
list_sample_lens_holders = list(lens_user_sample.owned_by)
list_sample_lens_holders = [x.lower() for x in list_sample_lens_holders]

for event_index, event in enumerate(poap_events):
    event_lens_user_holders = frozenset(event['holders']).intersection(list_sample_lens_holders)
    event_count_lens_users = len(event_lens_user_holders)
    poap_events[event_index]["count_lens_users"] = event_count_lens_users

In [21]:
poap_events_df = pd.DataFrame(poap_events)
poap_events_df = poap_events_df.drop(columns=['holders'])
top_10_poap_held_by_lens_users = poap_events_df.sort_values(by=['count_lens_users'], ascending=False).iloc[:10]
top_10_poap_held_by_lens_users['event_id'] = top_10_poap_held_by_lens_users['event_id'].astype('int64')
top_10_poap_held_by_lens_users

,event_id,token_count,count_lens_users
21909,66622,764,11
1097,12313,1383,11
19802,60152,147,10
15494,48089,146,10
5552,22504,316,10
17580,54012,279,10
17246,53108,89,9
17689,5437,162,9
17342,53358,139,9
2124,14512,818,9


In [22]:
events_df.join(top_10_poap_held_by_lens_users.set_index('event_id'), on='id', how='right')

,id,fancy_id,name,event_url,image_url,country,city,description,year,start_date,end_date,expiry_date,from_admin,virtual_event,event_template_id,event_host_id,private_event,token_count,count_lens_users
10752,66622,zapper-community-call-39-2022,Zapper Community Call #39,https://zapper.fi,https://assets.poap.xyz/zapper-community-call-...,,,This is a POAP badge as proof you attended Zap...,2022,22-Sep-2022,22-Sep-2022,22-Oct-2022,False,True,0,0,False,764,11
64627,12313,foundations-dao-general-2021,Foundations DAO General,https://daohaus.club/,https://assets.poap.xyz/foundations-dao-genera...,,,Foundations DAO is the largest permissioned da...,2021,31-Oct-2021,31-Oct-2021,01-Dec-2021,False,False,0,0,False,1383,11
16861,60152,zendfriends-ama-series-clearpool-150822-2022,ZEND&FRIENDS AMA Series: Clearpool 15/08/22,https://zklend.com/,https://assets.poap.xyz/zendfriends-ama-series...,,,The fifth episode of ZEND&FRIENDS AMA Series 0...,2022,15-Aug-2022,15-Aug-2022,15-Sep-2022,False,True,0,0,False,147,10
28576,48089,first-zkex-trivia-competition-2022,First ZKEX Trivia Competition,,https://assets.poap.xyz/first-zkex-trivia-comp...,,,This drop is for participating in the ZKEX Tri...,2022,06-Jun-2022,06-Jun-2022,06-Jul-2022,False,True,0,0,False,146,10
54284,22504,developer-dao-season-0-extension-vote-2022,Developer DAO - Season 0 Extension Vote,https://www.developerdao.com,https://assets.poap.xyz/developer-dao-season-0...,,,This POAP is rewarded to DAO members who voted...,2022,08-Jan-2022,08-Jan-2022,08-Feb-2022,False,True,0,0,False,316,10
22949,54012,pooltogether-community-call-67-2022,PoolTogether Community Call 67,https://pooltogether.com/discord,https://assets.poap.xyz/pooltogether-community...,,,Awarded to attendees of PoolTogether's Communi...,2022,08-Jul-2022,08-Jul-2022,08-Aug-2022,False,False,0,0,False,279,10
23799,53108,banklessdao-community-call-59-2022,BanklessDAO Community Call #59,,https://assets.poap.xyz/banklessdao-community-...,,,Awarded for attending the BanklessDAO Communit...,2022,01-Jul-2022,02-Jul-2022,01-Aug-2022,False,True,0,0,False,89,9
71421,5437,the-great-giveaway-1-sponsored-by-michitodd-2021,The GREAT GIVEAWAY #1 - Sponsored by @michi_todd,https://twitter.com/michi_todd,https://assets.poap.xyz/the-great-giveaway-1-s...,,,This POAP is a lottery ticket received during ...,2021,10-Aug-2021,10-Aug-2021,10-Oct-2021,False,True,0,0,False,162,9
23490,53358,developer-dao-vote-p15-2022,Developer DAO - Vote P15,https://www.developerdao.com/,https://assets.poap.xyz/developer-dao-vote-p15...,,,This POAP is rewarded to Developer DAO members...,2022,04-Jul-2022,04-Jul-2022,04-Aug-2022,False,False,0,0,False,139,9
60448,14512,thales-royale-poap-2021,Thales Royale POAP,https://thalesmarket.io/,https://assets.poap.xyz/thales-royale-poap-202...,,,This POAP commemorates the first edition of Th...,2021,02-Dec-2021,02-Dec-2021,25-Dec-2021,False,True,0,0,False,818,9


As we can see, results are totally different with a small sample (approximatively 5% of Lens users). This can be used to leverage clusterization model based on sample or on Badges & Poap holdings.

### 2 - Designing a system to provide personalized content recommendations on Lens

Without writing any code, design a system for providing personalized content recommendations on Lens. You are free to make assumptions about what data is available, but make sure you make the assumptions explicit. Please describe the data you would use, the kind of model you employ and why, how you would train that model, how you would assess the model, and how you would serve the model. Include as much detail as you feel is appropriate about the pros and cons of each of the decisions you make about the design






The recommendations provided online are generally designed to serve a certain purpose and create a certain value, either for the consumer, the supplier, another stakeholder such as an article producer, or several of them in parallel.
It is for me necessary to set the scene : Lens is a decentralized social graph protocol that enable users to set connection betweet each other, interests or any decentralized content. What we call web3 can be see in the future as allowing all users to own their content (social graph, milestones, contents) and this is exactly what web2 was missing for enable curative personalized algorithm.

Let's have a look at what data we could leverage : 

- **NFTs** : Web3 users can often be identified by the NFT they minted or acquired. They mostly have a sense of belonging to a community : NFT collections(Bored Apes, Azuki, CloneX), DAO belonging (KaizenDAO, gmDAO, ...). We can also see this as an opportunity to creates boundings between users, similar DAOs, similar contents...

- **Badges** : Can be Poap, or even more interesting : ZkBadges. These could be used as proof of attendance for specific events, but also delivered as "milestones" in our web3 environment (e.g 5k Txs on Uniswap, 500 Mints, ...). Again we can leverage this system to create users clusters by computing similarities and propose new users to follow, or even new web3 activities directly.

- **Lens Graph** : User's social content : Lenster's activity with collected content, follows, like... Will be one of the main source of inspiration for Data Science in a web3 environment.

What is great with these data is that they are all available directly OnChain ! I think the data retrieval process could be really improved using a dedicated Indexer & Data Pipeline to fasten the feeding time. At this time it is quite unstable the way to index and transform the data, but it will be a massive challenge : provide the data in the simpliest manner but with wide usage in the meantime because multiple algorithm with multiple Pipeline can be used, ev en more if Lens allows anyone to run its own algorithm.

![alt text](web3user.png "Web3 User's Content")

We want to build personalized recommendation, however the spectrum is pretty wide since we could recommend anything in any environment : 
- OnChain Activities
- NFTs / DAOs
- Lens application contents (Lenster, Lensfrens,  Orb, ...)

We will therefore focus on the latter : Lens content environment : Posts, follows and activities.



### 3 - SQL Query to map 1-1 addresses to profile_ids

A single wallet can own many Lens profile NFTs. Sometimes, we need to be able to identify which is the “default” or “canonical” profile for the wallet, i.e., the profile that is actually being used on Lens frontends. Write a SQL query that one-to-one maps addresses to profile_ids using the logic described below. The query should produce a table with two columns: address and profile_id. The only table you should need is the profiles table, which is described below.

-  Lens users have the option of setting a default profile explicitly. If is_default is true, select that profile as the default profile. Assume that only one profile that a wallet owns can be marked is_default.
-  There are many cases, however, where a default profile is not selected for a wallet. In these cases, we pick the profile that was minted first according to the timestamp associated with the profile as the default profile.


In [23]:
lens_user_sample.head()

,profile_id,owned_by,is_default,timestamp
0,0x9fef,0x6FB3af602BBdc75C03d1C5A442E2637615CC9b4f,False,2022-07-12 19:37:40.000000 UTC
1,0x01346a,0x7eF69546D56c2F7c499aC872E19cd000f82CFDc8,False,2022-10-01 05:03:48.000000 UTC
2,0x01346f,0xB85dC382fE75c161FDCa9D2f99CdCc4CD4babE92,False,2022-10-01 05:11:31.000000 UTC
3,0x68ab,0xe9FC2311a56cC11aa70039C39C908709e11cef74,False,2022-06-09 13:42:21.000000 UTC
4,0x013738,0xEf5c845D39Eb608C88b742eFd5739063eDC25888,False,2022-10-02 03:53:39.000000 UTC


**Query**

```sql{}
SELECT p1.owned_by as address, p1.profile_id
FROM profiles p1
WHERE p1.is_default=true OR (timestamp=(
	SELECT MIN(p2.timestamp)
	FROM profiles p2
	WHERE p2.owned_by=p1.owned_by
) AND NOT EXISTS(SELECT * FROM profiles p3 WHERE p3.is_default=1 AND p3.owned_by=p1.owned_by))
GROUP BY p1.owned_by, p1.profile_id;
```

Multiple answers are correct here, however I found this one the most efficient .